## 匯入套件和資料

In [1]:
import os
import jieba
import numpy as np
import csv
import pandas as pd

In [2]:
def csv_create(file_name,model,co_id,list_name):
    with open(file_name,model,newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['公司名稱','95,96年','96,97年','97,98年','98,99年','99,100年','100,101年','101,102年','102,103年','103,104年','104,105年','105,106年','106,107年','107,108年','108,109年'])
        writer.writerow([co_id]+list_name)

In [3]:
all_file = []
txt_root = 'C:\\Users\\DC\\中文詞向量\\1909文字'

for root, dirs, files in os.walk(txt_root): 
    for file in files: 
        print(root + '\\' + file)
        all_file.append(root + '\\' + file)

C:\Users\DC\中文詞向量\1909文字\2006.txt
C:\Users\DC\中文詞向量\1909文字\2007.txt
C:\Users\DC\中文詞向量\1909文字\2008.txt
C:\Users\DC\中文詞向量\1909文字\2009.txt
C:\Users\DC\中文詞向量\1909文字\2010.txt
C:\Users\DC\中文詞向量\1909文字\2011.txt
C:\Users\DC\中文詞向量\1909文字\2012.txt
C:\Users\DC\中文詞向量\1909文字\2013.txt
C:\Users\DC\中文詞向量\1909文字\2014.txt
C:\Users\DC\中文詞向量\1909文字\2015.txt
C:\Users\DC\中文詞向量\1909文字\2016.txt
C:\Users\DC\中文詞向量\1909文字\2017.txt
C:\Users\DC\中文詞向量\1909文字\2018.txt
C:\Users\DC\中文詞向量\1909文字\2019.txt
C:\Users\DC\中文詞向量\1909文字\2020.txt


In [4]:
all_txt = []

for i in range(0,len(all_file)):
    txt = open(all_file[i], 'r', encoding='utf-8').read()
    txt = txt.replace('■','').replace('、','').replace('：','').replace('\n','').replace(' ','').replace('●','').replace('，','').replace('│','').replace('。','').replace(',','').replace(';','').replace('(','').replace(')','').replace('；','')
    a =jieba.lcut(txt)
    all_txt.append(a)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\DC\AppData\Local\Temp\jieba.cache
Loading model cost 0.435 seconds.
Prefix dict has been built successfully.


## 載入模型後將文字轉為向量！

In [5]:
from gensim.models import word2vec
model = word2vec.Word2Vec.load('word300.model') #載入模型

In [6]:
padding = np.zeros((model.vector_size,), dtype=np.float32) #0向量
all_vec_add = []
vec_add = 0

for j in range(0,len(all_txt)):
    for i in all_txt[j]:
        try:
            vec_add += model.wv[i]
        except:
            model.wv[i] = padding
            vec_add += model.wv[i]
    all_vec_add.append(vec_add)
    vec_add = 0
    
print(all_vec_add[1])

[ 3.50088425e+01  4.77968178e+01  1.41553736e+01 -4.53406525e+01
  2.59528675e+01  2.83755894e+01  7.65147934e+01  4.87334709e+01
 -5.18335991e+01  2.21948051e+01 -2.68511925e+01  2.58889656e+01
 -6.53080063e+01 -6.99132462e+01 -3.89551239e+01 -3.55277939e+01
 -1.07396797e+02 -1.62416327e+00 -3.31893849e+00  2.90915146e+01
  2.85854549e+01 -7.57651377e+00  9.65745621e+01  4.73824463e+01
 -7.36086578e+01  2.44867725e+01  9.20202923e+00  4.50790672e+01
 -1.36391211e+01  1.98512936e+01 -3.02513981e+01 -3.42723770e+01
 -1.02968712e+02  6.28772240e+01  4.36986389e+01  8.15570374e+01
 -7.89771576e+01  2.15123520e+01 -3.98312569e+01  1.96262531e+01
 -2.54308724e+00  4.16065750e+01 -4.58424606e+01 -2.94210148e+01
  3.22047920e+01 -5.94070005e+00 -1.20474939e+01 -6.01595603e-02
 -4.35034065e+01  1.11294308e+01  6.43957443e+01 -6.51241226e+01
 -5.99888763e+01 -1.55930605e+01 -3.07176208e+01  3.90762138e+01
  2.11125088e+00 -8.30321960e+01 -3.74248352e+01  3.03004246e+01
  2.99406281e+01 -7.01422

## 歐式計算距離寫入csv

In [8]:
#歐式
Euc = []

for i in range(0,len(all_vec_add)-1):
    x = all_vec_add[i]
    y = all_vec_add[i+1]
    dis = np.linalg.norm( all_vec_add[i] - all_vec_add[i+1])
    Euc.append(dis)
    print(dis)

54.133915
43.062218
287.33676
45.87058
59.09678
28.84357
349.75732
1684.4163
117.87473
360.80096
143.11743
173.32533
166.95586
281.5997


In [45]:
csv_file = 'euc_per_year.csv'
write_mode = 'w'
co_id = '1909'

csv_create(csv_file,write_mode,co_id,Euc)

## 餘弦計算距離寫入csv

In [7]:
#餘弦
Man = []

for i in range(0,len(all_vec_add)-1):
    x = all_vec_add[i]
    y = all_vec_add[i+1]
    
    #小數方式
    #np.set_printoptions(precision=10,suppress=True) #位數
    #dis =  np.array([1 - np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))])
    
    #科學記號方式
    dis =  1 - np.dot(x,y)/(np.linalg.norm(x)*np.linalg.norm(y))
    print(dis)
    Man.append(dis)

3.5762786865234375e-07
5.960464477539062e-07
1.6689300537109375e-06
8.940696716308594e-07
6.556510925292969e-07
4.76837158203125e-07
1.0132789611816406e-06
7.152557373046875e-07
4.172325134277344e-07
2.384185791015625e-07
5.960464477539063e-08
2.384185791015625e-07
2.384185791015625e-07
2.384185791015625e-07


In [32]:
csv_file = 'man_per_year.csv'
write_mode = 'w'
co_id = '1909'

csv_create(csv_file,write_mode,co_id,Man)

### 其他

In [75]:
similarity = model.wv.similarity('股東',all_txt[0][2]) #兩點距離
similarity

0.9796264

In [12]:
for item in model.wv.most_similar('營收'):
    print(item)

('億元', 0.9998481273651123)
('年', 0.9998316168785095)
(',', 0.9998290538787842)
('，', 0.9998282790184021)
('併', 0.9998176693916321)
('新', 0.9998173117637634)
('年', 0.9998149871826172)
('為', 0.9998137950897217)
('較', 0.9998124241828918)
('萬噸', 0.9998113512992859)


In [ ]:
#https://www.itread01.com/content/1548060671.html